In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
import requests
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

In [3]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str)-> str:
    """This function fetches the currency conversion factor from given base currency to target currency"""
    url = f"https://v6.exchangerate-api.com/v6/a411f1bec9afb8ef0ef05dd6/pair/{base_currency}/{target_currency}"

    response = requests.get(url)
    return response.json()

In [4]:
@tool
def convert(base_currency_value: int, conversion_rate: float)-> float:
    """Given a currency conversion rate this function calculates the target currency value from a base currency value"""
    return base_currency_value * conversion_rate

In [5]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1747440002,
 'time_last_update_utc': 'Sat, 17 May 2025 00:00:02 +0000',
 'time_next_update_unix': 1747526402,
 'time_next_update_utc': 'Sun, 18 May 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.6035}

In [6]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
)

# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     temperature=0.7,
# )

In [8]:
llm_tool = llm.bind_tools([get_conversion_factor, convert])
llm_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000237320653C0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000237323EFC70>, root_client=<openai.OpenAI object at 0x0000023732064AC0>, root_async_client=<openai.AsyncOpenAI object at 0x0000023732064BE0>, model_name='gpt-4o-mini', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetches the currency conversion factor from given base currency to target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'description': 'Given a currency conversion rate this function calculates the target currency value fro

In [24]:
llm_tool.invoke('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uAETeBfsLgTQlVIq83DrWa5l', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_ES8qxPpoguad84BJglQhTLdZ', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 121, 'total_tokens': 181, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BY5e73fN1QhjF2Z98zFndyTEYWAWF', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--5e37ef86-5079-4436-8e30-ea9e87205828-0', tool_calls=[

In [25]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [26]:
# --- Step 6: Run the Agent ---
user_query = "What is the conversion factor from USD to INR, and based on that can you convert 10 USD to INR"

response = agent_executor.invoke({"input": user_query})
response



> Entering new AgentExecutor chain...
Thought: I need to first find the conversion factor from USD to INR and then use that factor to convert 10 USD to INR. 

Action:
```
{
  "action": "get_conversion_factor",
  "action_input": {
    "base_currency": "USD",
    "target_currency": "INR"
  }
}
```

Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1747440002, 'time_last_update_utc': 'Sat, 17 May 2025 00:00:02 +0000', 'time_next_update_unix': 1747526402, 'time_next_update_utc': 'Sun, 18 May 2025 00:00:02 +0000', 'base_code': 'USD', 'target_code': 'INR', 'conversion_rate': 85.6035}
Thought:Now that I have the conversion factor of 85.6035 from USD to INR, I can convert 10 USD to INR using this rate.

Action:
```
{
  "action": "convert",
  "action_input": {
    "base_currency_value": 10,
    "conversion_rate": 85.6035
  }
}
```

Observation: 856.035
Thought:I have s

{'input': 'What is the conversion factor from USD to INR, and based on that can you convert 10 USD to INR',
 'output': 'The conversion factor from USD to INR is 85.6035. Therefore, 10 USD is equivalent to 856.035 INR.'}